# Code minimal nécessaire pour Query

In [1]:
DATA_FOLDER = ""
XML_INITIAL_FILE = ""
ANTI_DICT_FILE = ""
STEMMED_REPLACEMENTS = ""

In [ ]:
import pandas

substitutions = pandas.read_csv(STEMMED_REPLACEMENTS, sep="\t", header=None, na_filter=False)
substitutions.head(3)  # Fichier de Substitutions par lemmes

In [ ]:
from index import Corpus

with open(XML_INITIAL_FILE, "r", encoding="utf-8") as f:
    CORPUS = Corpus.model_validate_xml(f.read())

In [ ]:
from index import InvertedIndex

replacements = {inp: repl for inp, repl in zip(substitutions[0], substitutions[1])}

INDEX = {}
for zone in ["titre", "texte", "legendes"]:
    INDEX[zone] = InvertedIndex()
    
    # TODO : Construire l'index en remplaçant les tokens par leurs lemmes à posteriori
    # TODO : 1. Construire un nouvel objet InvertedIndex 2. Récupérer chaque liste de document associé au token 3. Rassembler les doc_id qui ont le même lemme
    
    for token, doc_ids in CORPUS.inverted_token_index(zones=[zone]).items():
        
        filtered_token = replacements.get(token, token)
        
        if filtered_token not in INDEX[zone]:
            INDEX[zone][filtered_token] = []
            
        INDEX[zone][filtered_token].extend(doc_ids)
        
INDEX["texte"].keys()

In [ ]:
from index import Query
q = Query.build('montre moi les articles qui parlent de environnement minimum depuis juillet 2011 et qui nest pas dans la rubrique Focus')
print(q)
q.search(documents=CORPUS, index=INDEX)